In [75]:
import pandas as pd, sqlite3
conn = sqlite3.connect("../data/checking-logs.sqlite")


In [76]:
## Viewing table schema test
pd.read_sql("PRAGMA table_info(test);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [77]:
## First 10 rows of the test table
pd.read_sql("SELECT * FROM test LIMIT 10;", conn)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [107]:
## Minimum difference (in hours) before deadline
query_min = """
SELECT
    c.uid,
    MIN((julianday(d.deadlines,'unixepoch') - julianday(c.first_commit_ts)) * 24) AS delta_hours
FROM test c
JOIN deadlines d ON c.labname = d.labs
WHERE c.labname != 'project1'
GROUP BY c.uid
ORDER BY delta_hours ASC
LIMIT 1;
"""
df_min = pd.read_sql(query_min, conn)


In [109]:
## Maximum difference value
query_max = """
SELECT
    c.uid,
    MAX((julianday(d.deadlines,'unixepoch') - julianday(c.first_commit_ts)) * 24) AS delta_hours
FROM test c
JOIN deadlines d ON c.labname = d.labs
WHERE c.labname != 'project1'
GROUP BY c.uid
ORDER BY delta_hours DESC
LIMIT 1;
"""
df_max = pd.read_sql(query_max, conn)


In [112]:
## Average difference (without uid)
query_avg = """
SELECT
    AVG((julianday(c.first_commit_ts) - julianday(d.deadlines,'unixepoch')) * 24) AS avg_delta_hours
FROM test c
JOIN deadlines d ON c.labname = d.labs
WHERE c.labname != 'project1';
"""
df_avg = pd.read_sql(query_avg, conn)


In [115]:
## Creating views_diff: uid, avg_diff, pageviews
query_views_diff = """
SELECT
    c.uid,
    AVG((julianday(c.first_commit_ts) - julianday(d.deadlines,'unixepoch')) * 24) AS avg_diff,
    COUNT(p.datetime) AS pageviews
FROM test c
LEFT JOIN deadlines d ON c.labname = d.labs
LEFT JOIN pageviews p ON c.uid = p.uid
WHERE c.labname != 'project1'
GROUP BY c.uid;
"""
views_diff = pd.read_sql(query_views_diff, conn)


In [90]:
## Correlation between number of views and time until deadline
views_diff.corr(numeric_only=True)


,avg_diff,pageviews
avg_diff,1.000000,0.185042
pageviews,0.185042,1.000000


In [108]:
df_min

,uid,delta_hours
0,user_25,2.867236


In [110]:
df_max

,uid,delta_hours
0,user_30,202.38473


In [113]:
df_avg

,avg_delta_hours
0,-89.687686


In [116]:
views_diff[['avg_diff','pageviews']].corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.185042
pageviews,-0.185042,1.000000


In [ ]:
conn.close()